In [182]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix
from sklearn import model_selection
from matplotlib import pyplot as plt
import copy
import math
import warnings
warnings.filterwarnings('ignore')

In [126]:
df1=pd.read_pickle('subscribers')

In [124]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227628 entries, 162502 to 598241
Data columns (total 29 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   subid                         227628 non-null  int64         
 1   package_type                  192054 non-null  object        
 2   num_weekly_services_utilized  117178 non-null  float64       
 3   preferred_genre               191302 non-null  object        
 4   intended_use                  224079 non-null  object        
 5   weekly_consumption_hour       189698 non-null  float64       
 6   num_ideal_streaming_services  115458 non-null  float64       
 7   retarget_TF                   227628 non-null  bool          
 8   age                           192459 non-null  float64       
 9   male_TF                       227359 non-null  object        
 10  country                       227628 non-null  object        
 11  attribut

In [26]:
df1.head()

,subid,package_type,num_weekly_services_utilized,preferred_genre,intended_use,weekly_consumption_hour,num_ideal_streaming_services,retarget_TF,age,male_TF,...,creation_until_cancel_days,cancel_before_trial_end,trial_end_date,initial_credit_card_declined,revenue_net,join_fee,language,paid_TF,refund_after_trial_TF,payment_type
162502,21724479,economy,NaN,comedy,access to exclusive content,NaN,NaN,False,NaN,False,...,NaN,True,2020-02-08,False,NaN,NaN,ar,True,False,Standard Charter
380374,23383224,base,NaN,comedy,access to exclusive content,22.951579,NaN,False,70.0,False,...,0.0,False,2020-03-16,False,0.0,0.3303,ar,False,False,NaN
496617,26844789,enhanced,3.362885,regional,replace OTT,36.001186,1.951776,False,25.0,True,...,1.0,False,2019-12-22,False,0.0,0.1101,ar,False,False,NaN
356799,29417030,base,NaN,drama,replace OTT,20.051667,NaN,False,30.0,False,...,0.0,False,2020-02-11,False,0.0,0.0367,ar,False,False,NaN
681701,26723159,base,3.731089,comedy,replace OTT,22.951579,2.506384,False,28.0,False,...,9.0,False,2019-10-20,False,0.0,0.0367,ar,False,False,NaN


In [119]:
df2=pd.read_pickle('engagement')

In [118]:
df3=pd.read_pickle('customer_service_reps')

## A/B Testing

In [39]:
df1.plan_type.value_counts()

base_uae_14_day_trial                227096
high_uae_14_day_trial                   325
low_uae_no_trial                        167
base_eur_14_day_trial                    18
high_sar_14_day_trial                    12
low_gbp_14_day_trial                      4
high_aud_14_day_trial                     2
low_eur_no_trial                          1
base_uae_no_trial_7_day_guarantee         1
low_sar_no_trial                          1
high_jpy_14_day_trial                     1
Name: plan_type, dtype: int64

In [121]:
df1.monthly_price.value_counts()

4.7343    227101
5.1013       325
1.0643       167
4.4407        18
4.3673        12
4.0003         2
1.1744         1
0.8074         1
4.6976         1
Name: monthly_price, dtype: int64

In [122]:
df_low

,subid,package_type,num_weekly_services_utilized,preferred_genre,intended_use,weekly_consumption_hour,num_ideal_streaming_services,retarget_TF,age,male_TF,...,creation_until_cancel_days,cancel_before_trial_end,trial_end_date,initial_credit_card_declined,revenue_net,join_fee,language,paid_TF,refund_after_trial_TF,payment_type
511887,22031615,NaN,3.952015,NaN,NaN,NaN,NaN,True,25.0,False,...,22.0,True,2019-07-24,True,0.0000,0.0000,ar,False,False,NaN
31201,28496166,NaN,2.273003,NaN,NaN,NaN,NaN,True,40.0,False,...,NaN,True,2019-07-27,False,3.0828,0.0000,ar,True,False,Paypal
522051,21941388,NaN,NaN,NaN,NaN,NaN,NaN,True,26.0,False,...,0.0,False,2019-07-28,False,0.0000,0.0000,ar,False,False,NaN
12672,21827870,NaN,2.361372,NaN,NaN,NaN,NaN,True,49.0,False,...,NaN,True,2019-07-25,False,1.0276,0.0000,ar,True,False,Standard Charter
124600,21327523,NaN,4.231850,NaN,NaN,NaN,NaN,True,41.0,True,...,NaN,True,2019-07-26,False,3.0828,0.0000,ar,True,False,Paypal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669347,24078393,enhanced,2.094663,comedy,other,28.751405,1.762910,False,20.0,False,...,4.0,False,2019-11-10,False,0.0000,0.0367,ar,False,False,NaN
355551,25953928,base,2.331916,comedy,replace OTT,25.851492,1.658939,False,48.0,False,...,6.0,False,2019-08-23,False,0.0000,0.0367,ar,False,False,NaN
64703,29467306,enhanced,NaN,comedy,access to exclusive content,30.201361,NaN,False,53.0,False,...,NaN,True,2020-02-14,False,4.2572,0.0367,ar,True,False,Standard Charter
7746,25226506,NaN,NaN,NaN,NaN,NaN,NaN,True,50.0,True,...,12.0,True,2019-07-26,False,0.0000,0.0000,ar,True,True,Paypal


In [71]:
df_base = pd.concat([df1[df1['plan_type']=='base_uae_14_day_trial'],df1[df1['plan_type']=='base_eur_14_day_trial'],df1[df1['plan_type']=='base_uae_no_trial_7_day_guarantee']],axis=0)

In [73]:
len(df_base)

227115

In [74]:
df_low = pd.concat([df1[df1['plan_type']=='low_uae_no_trial'],df1[df1['plan_type']=='low_gbp_14_day_trial'],df1[df1['plan_type']=='low_eur_no_trial'],df1[df1['plan_type']=='low_sar_no_trial']],axis=0)

In [76]:
len(df_low)

173

In [120]:
len(df_high)

340

In [77]:
df_high = pd.concat([df1[df1['plan_type']=='high_uae_14_day_trial'],df1[df1['plan_type']=='high_sar_14_day_trial'],df1[df1['plan_type']=='high_aud_14_day_trial'],df1[df1['plan_type']=='high_jpy_14_day_trial']],axis=0)

In [79]:
df1.paid_TF.value_counts()

False    135578
True      92050
Name: paid_TF, dtype: int64

In [82]:
df_base['paid_TF'].mean()

0.4041476784888713

In [83]:
df_low['paid_TF'].mean()

0.791907514450867

In [85]:
df_high['paid_TF'].mean()

0.36764705882352944

In [179]:
df1.paid_TF.value_counts()

False    135578
True      92050
Name: paid_TF, dtype: int64

In [47]:
pd.crosstab(df1['paid_TF'],df1['refund_after_trial_TF'])

refund_after_trial_TF,False,True
paid_TF,,
False,135578,0
True,82805,9245


In [117]:
#H0: p_base = p_low
#Ha: p_base < p_low
t = (df_base['paid_TF'].mean()- df_low['paid_TF'].mean()) / np.sqrt((df_base['paid_TF'].var() / len(df_base) + df_low['paid_TF'].var() / len(df_low)))
print(t)

-12.520493014787231


In [114]:
t2 = (df_base['paid_TF'].mean() - df_high['paid_TF'].mean()) / np.sqrt((df_base['paid_TF'].var() / len(df_base) + df_high['paid_TF'].var() / len(df_high)))
print(t2)

1.3927375670609472


In [113]:
p_base = df_base['paid_TF'].mean()
p_low = df_low['paid_TF'].mean()
t_a = 1.96
t_b = 0.84
p_mean = df1['paid_TF'].mean()
min_effect = 0.03
n = ((t_a) * np.sqrt(2 * p_mean * (1 - p_mean)) + t_b * np.sqrt( p_base * (1 - p_base) + p_low * (1 - p_low))) ** 2 / (min_effect * min_effect)
n = int(n) + 1
print(n)

3992


In [115]:
p_base = df_base['paid_TF'].mean()
p_high = df_high['paid_TF'].mean()
t_a = 1.96
t_b = 0.84
p_mean = df1['paid_TF'].mean()
min_effect = 0.03
n = ((t_a) * np.sqrt(2 * p_mean * (1 - p_mean)) + t_b * np.sqrt( p_base * (1 - p_base) + p_high * (1 - p_high))) ** 2 / (min_effect * min_effect)
n = int(n) + 1
print(n)

4175


In [101]:
sample = []
for i in range(10):
    sample_base =  df_base.sample(n)
    sample_low =  df_low.sample(173)
    t1 = (sample_low['paid_TF'].mean() - sample_base['paid_TF'].mean()) / np.sqrt((sample_base['paid_TF'].var()+sample_low['paid_TF'].var()) / n)
    #t2 = (sampleB.mean() - sampleA.mean() - min_effect) / numpy.sqrt((sampleA.var()+sampleB.var()) / n)
    sample.append([sample_base, sample_low])
    if t1 > t_a:
        print('reject H0')
    else:
        print('accept H0')

reject H0
reject H0
reject H0
reject H0
reject H0
reject H0
reject H0
reject H0
reject H0
reject H0


In [108]:
boundary_wald_low = np.log(0.2 / (1 - 0.05))
boundary_wald_high = np.log((1 - 0.2) / 0.05)
boundary_conservative_low = np.log(0.2)
boundary_conservative_high = np.log(1 / 0.05)
f_1 = np.log(p_low / p_base)
f_0 = np.log((1 - p_low) / (1 - p_base))
count_list = []
for j in range(10):
    summary = 0
    count = 0
    sample_A = sample[j][0]
    sample_B = sample[j][1]
    sample_combine = pd.concat([sample_base, sample_low])
    sample_random = sample_combine.sample(frac=1.0)
    for i in range(len(sample_random)):
        count += 1
        if sample_random['paid_TF'].values[i] == 1:
            summary += f_1
            if summary > boundary_wald_high:
                print('stop early, reject H0')
                count_list.append(count)                
                break
        else:
            summary += f_0
            if summary < boundary_wald_low:
                print('stop early, accept H0')
                count_list.append(count) 
                break
        if count == len(sample_random):
            print('use the full sample')
            count_list.append(count)
print(count_list)

stop early, accept H0
stop early, accept H0
stop early, accept H0
stop early, accept H0
stop early, accept H0
stop early, accept H0
stop early, accept H0
stop early, accept H0
stop early, accept H0
stop early, accept H0
[9, 2, 4, 15, 4, 9, 4, 5, 2, 5]


## Attribution

In [157]:
df1.attribution_survey.value_counts()

facebook            119126
tv                   39904
referral             20882
search                8492
pinterest             7856
other                 6496
public_radio          6219
social_organic        3869
youtube               3108
podcast               2995
affiliate             1962
ott                   1443
sirius xm             1122
facebook_organic       765
direct_mail            408
amfm_radio             337
Name: attribution_survey, dtype: int64

In [164]:
df1.attribution_technical.value_counts()

facebook                   80251
email                      25690
search                     25306
organic                    22013
brand sem intent google    18524
google_organic             10691
affiliate                   9894
email_blast                 7277
pinterest                   6065
referral                    5170
facebook_organic            3272
discovery                   2571
brand sem intent bing       2231
other                       1786
display                     1407
bing                        1146
internal                    1122
podcast                      985
youtube                      913
bing_organic                 369
vod                          297
ott                          158
direct_mail                  139
quora                        100
samsung                       86
appstore                      44
criteo                        44
pinterest_organic             30
influencer                    21
playstore                     12
content_gr

In [148]:
df_ad=pd.read_excel('advertising_spend_data.xlsx',header=0,names=None)

In [149]:
df_ad.columns = list(df_ad.loc[1])
df_ad = df_ad.drop([0])
df_ad = df_ad.drop([1])


In [150]:
df_ad

,date,facebook,email,search,brand sem intent google,affiliate,email_blast,pinterest,referral,sum
2,2019-06-30 00:00:00,84800,75000,17300,21400,11900,10500,5900,6100,232900
3,2019-07-31 00:00:00,60000,75600,17300,21300,12000,29800,6000,6600,228600
4,2019-08-31 00:00:00,52300,76100,18100,21300,16300,65900,6200,6800,263000
5,2019-09-30 00:00:00,49000,75800,18900,21300,16400,73700,6000,7400,268500
6,2019-10-31 00:00:00,51300,76000,21000,21400,12200,32300,6700,7300,228200
7,2019-11-30 00:00:00,56100,75900,21400,21600,11700,9800,6300,7300,210100
8,2019-12-31 00:00:00,51400,76100,32100,22200,12100,3500,7200,8000,212600
9,2020-01-31 00:00:00,49000,75500,33600,22700,14300,300,6700,7400,209500
10,2020-02-29 00:00:00,49700,76600,30100,21700,15000,300,6400,6500,206300
11,2020-03-31 00:00:00,49100,78000,22700,21200,17600,300,5900,10000,204800


In [177]:
df_ad['sum'].mean()

226450.0

In [178]:
df_ad['sum'].sum()

2264500

In [154]:
cost_facebook = df_ad['facebook'].mean()
cost_email = df_ad['email'].mean()
cost_search = df_ad['search'].mean()
cost_google = df_ad['brand sem intent google'].mean()
cost_affiliate = df_ad['affiliate'].mean()
cost_email_blast = df_ad['email_blast'].mean()
cost_pinterest = df_ad['pinterest'].mean()
cost_referral = df_ad['referral'].mean()




#### Last Interaction

In [173]:
CAC_facebook = cost_facebook / len(df1[df1.attribution_technical == 'facebook'])
print('facebook: %.2f'%CAC_facebook)
CAC_email =  cost_email / len(df1[df1.attribution_technical == 'email'])
print('email: %.2f'%CAC_email)
CAC_search = cost_search/ len(df1[df1.attribution_technical == 'search'])
print('search: %.2f'%CAC_search)
CAC_google = cost_google/ len(df1[df1.attribution_technical == 'brand sem intent google'])
print('google: %.2f'%CAC_google)
CAC_affiliate = cost_affiliate/ len(df1[df1.attribution_technical == 'affiliate'])
print('affiliate: %.2f'%CAC_affiliate)
CAC_email_blast = cost_email_blast/ len(df1[df1.attribution_technical == 'email_blast'])
print('email_blast: %.2f'%CAC_email_blast)    
CAC_pinterest = cost_pinterest/ len(df1[df1.attribution_technical == 'pinterest'])
print('pinterest: %.2f'%CAC_pinterest)
CAC_referral = cost_referral/ len(df1[df1.attribution_technical == 'referral'])
print('referral: %.2f'%CAC_referral)

facebook: 0.69
email: 2.96
search: 0.92
google: 1.17
affiliate: 1.41
email_blast: 3.11
pinterest: 1.04
referral: 1.42


#### First Interaction

In [176]:
CAC_facebook = cost_facebook / len(df1[df1.attribution_survey == 'facebook'])
print('facebook: %.2f'%CAC_facebook)
CAC_email =  cost_email / len(df1[df1.attribution_survey == 'email'])
print('email: %.2f'%CAC_email)
CAC_search = cost_search/ len(df1[df1.attribution_survey == 'search'])
print('search: %.2f'%CAC_search)
CAC_google = cost_google/ len(df1[df1.attribution_survey == 'brand sem intent google'])
print('google: %.2f'%CAC_google)
CAC_affiliate = cost_affiliate/ len(df1[df1.attribution_survey == 'affiliate'])
print('affiliate: %.2f'%CAC_affiliate)
CAC_email_blast = cost_email_blast/ len(df1[df1.attribution_survey == 'email_blast'])
print('email_blast: %.2f'%CAC_email_blast)    
CAC_pinterest = cost_pinterest/ len(df1[df1.attribution_survey == 'pinterest'])
print('pinterest: %.2f'%CAC_pinterest)
CAC_referral = cost_referral/ len(df1[df1.attribution_survey == 'referral'])
print('referral: %.2f'%CAC_referral)

facebook: 0.46
email: inf
search: 2.74
google: inf
affiliate: 7.11
email_blast: inf
pinterest: 0.81
referral: 0.35


## Churn

In [187]:
df1['convert_T/F'] = np.where((df1['paid_TF']==True) & (df1['refund_after_trial_TF']==False), True, False) 

In [199]:
df1['male_TF'] = np.where(df1['male_TF']==True,1,0)
df1['retarget_TF'] = np.where(df1['retarget_TF']==True,1,0)

In [211]:
df1.isnull().sum()

subid                                0
package_type                     35574
num_weekly_services_utilized    110450
preferred_genre                  36326
intended_use                      3549
weekly_consumption_hour          37930
num_ideal_streaming_services    112170
retarget_TF                          0
age                              35169
male_TF                              0
country                              0
attribution_technical                0
attribution_survey                2644
op_sys                           13375
months_per_bill_period               0
plan_type                            0
monthly_price                        0
discount_price                       0
account_creation_date                0
creation_until_cancel_days       75860
cancel_before_trial_end              0
trial_end_date                       0
initial_credit_card_declined         0
revenue_net                      34904
join_fee                         34904
language                 

In [215]:
df_new=df1.fillna(df1.mean())

In [272]:
df_new.head()

,subid,package_type,num_weekly_services_utilized,preferred_genre,intended_use,weekly_consumption_hour,num_ideal_streaming_services,retarget_TF,age,male_TF,...,cancel_before_trial_end,trial_end_date,initial_credit_card_declined,revenue_net,join_fee,language,paid_TF,refund_after_trial_TF,payment_type,convert_T/F
162502,21724479,economy,2.998568,comedy,access to exclusive content,27.997724,1.999848,0,757.975457,0,...,True,2020-02-08,False,1.557353,0.115145,ar,True,False,Standard Charter,True
380374,23383224,base,2.998568,comedy,access to exclusive content,22.951579,1.999848,0,70.000000,0,...,False,2020-03-16,False,0.000000,0.330300,ar,False,False,NaN,False
496617,26844789,enhanced,3.362885,regional,replace OTT,36.001186,1.951776,0,25.000000,1,...,False,2019-12-22,False,0.000000,0.110100,ar,False,False,NaN,False
356799,29417030,base,2.998568,drama,replace OTT,20.051667,1.999848,0,30.000000,0,...,False,2020-02-11,False,0.000000,0.036700,ar,False,False,NaN,False
681701,26723159,base,3.731089,comedy,replace OTT,22.951579,2.506384,0,28.000000,0,...,False,2019-10-20,False,0.000000,0.036700,ar,False,False,NaN,False


In [216]:
df_train, df_test = model_selection.train_test_split(df_new, test_size=0.3)

In [217]:
y_train=df_train['convert_T/F']
x_train=df_train.drop('convert_T/F',axis=1)
y_test=df_test['convert_T/F']
x_test=df_test.drop('convert_T/F',axis=1)

In [233]:
x_col_name = ['weekly_consumption_hour', 'num_ideal_streaming_services','age','retarget_TF','male_TF']    # list of x covariates

In [226]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [234]:
lpm_model = sm.OLS(endog=df_train['convert_T/F'], exog=df_train[x_col_name])
lpm_results = lpm_model.fit()
print("\nRESULTS OF LOGIT MODEL FITTING\n")
print(lpm_results.summary())


RESULTS OF LOGIT MODEL FITTING

                                 OLS Regression Results                                
Dep. Variable:            convert_T/F   R-squared (uncentered):              -36772.418
Model:                            OLS   Adj. R-squared (uncentered):         -36773.572
Method:                 Least Squares   F-statistic:                         -3.187e+04
Date:                Fri, 16 Dec 2022   Prob (F-statistic):                        1.00
Time:                        21:17:08   Log-Likelihood:                     -1.0928e+05
No. Observations:              159339   AIC:                                  2.186e+05
Df Residuals:                  159334   BIC:                                  2.186e+05
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                                   coef    std err          t      P>|t|      [0.025   

In [235]:
logit_model = sm.Logit(endog=df_train['convert_T/F'], exog=df_train[x_col_name])
logit_results = logit_model.fit()
print("\nRESULTS OF LOGIT MODEL FITTING\n")
print(logit_results.summary())

Optimization terminated successfully.
         Current function value: 0.654494
         Iterations 4

RESULTS OF LOGIT MODEL FITTING

                           Logit Regression Results                           
Dep. Variable:            convert_T/F   No. Observations:               159339
Model:                          Logit   Df Residuals:                   159334
Method:                           MLE   Df Model:                            4
Date:                Fri, 16 Dec 2022   Pseudo R-squ.:                0.001408
Time:                        21:17:35   Log-Likelihood:            -1.0429e+05
converged:                       True   LL-Null:                   -1.0443e+05
Covariance Type:            nonrobust   LLR p-value:                 2.028e-62
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
weekly_consumption_hour          0.0143

In [265]:
def apply_offers(df, threshold, base_price, discount_amount, cases, acceptance_rate_churn, acceptance_rate_nonchurn):
    df_f = copy.deepcopy(df_train)

    # set offer parameters for all observations
    df_f['price_no_offer'] = base_price
    df_f.loc[df_f['convert_T/F'] == 0, 'price_no_offer'] = 0  # if the person churns they will pay 0

    # determine whether indiviudals accept offer
    df_f.loc[df_f['convert_T/F'] == 0, 'accept_offer_prob'] = acceptance_rate_churn  # those who were going to churn are less likely to want the offer than those who were going to stay
    df_f.loc[df_f['convert_T/F'] == 1, 'accept_offer_prob'] = acceptance_rate_nonchurn

    # make offer under lpm predictions for select threshold
    df_f['make_offer_lpm'] = (df_f['prob_churn_lpm'] > threshold)     # predict whether individual will churn or not
    df_f['accept_offer_lpm'] = (df_f['make_offer_lpm'] & df_f['would_accept_offer_rand_draw'])
    df_f['price_lpm_offer'] = df_f['price_no_offer']  # initialize prices from "do nothing case"
    df_f.loc[df_f['accept_offer_lpm'], 'price_lpm_offer'] = base_price * (1-discount_amount)    # for those who were both made an offer and would accept, set prices to discounted price

    # make offer under logit predictions for select threshold
    df_f['make_offer_logit'] = (df_f['prob_churn_logit'] > threshold)     # predict whether individual will churn or not
    df_f['accept_offer_logit'] = (df_f['make_offer_logit'] & df_f['would_accept_offer_rand_draw'])
    df_f['price_logit_offer'] = df_f['price_no_offer']  # initialize prices from "do nothing case"
    df_f.loc[df_f['accept_offer_logit'], 'price_logit_offer'] = base_price * (1-discount_amount)    # for those who were both made an offer and would accept, set prices to discounted price

    df_impact = pd.DataFrame(index=cases, columns=['subs', 'revenue', 'lift_from_no_offer'])
    for c in cases:
        df_impact.loc[c, 'subs'] = (df_f['price_' + c] > 0).sum()
        df_impact.loc[c, 'revenue'] = df_f['price_' + c].sum()
    for c in cases:
        df_impact.loc[c, 'lift_from_no_offer'] = df_impact.loc[c, 'revenue'] / df_impact.loc['no_offer', 'revenue'] - 1

    return df_f, df_impact